#### ¿Cual es la mejor inversión publicitaria y como optimizarla?
Se ha estado utilizando diferentes medios como canales de publicidad convirtiendose en una de las principales inversiones, el objetivo es determinar la estrategia más efectiva y rentable considerando los costos e ingresos para luego recomendar la asignación óptima para cada canal.

El conjunto de datos que almacena la información requerida se compone de la siguiente manera: los ingresos obtenidos en ventas junto con otras tres variables continuas que representan presupuestos invertidos en la TV, Radio y Redes sociales y una variable categórica que muestra el nivel de popularidad del o los influencers que participaron en la promoción. El objetivo se basará en desarrollar un modelo lineal que se ajuste a una sola variable independiente con la variable objetivo.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from plotly.subplots import make_subplots
import plotly.express as px
import plotly.graph_objects as go
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
from scipy.stats import pearsonr, zscore, kurtosis, norm, kstest
from sklearn.linear_model import LinearRegression


df = pd.read_csv("data/advertising_and_sales.csv")
df.set_index("id", inplace=True) # colocando a la columna 'id' como índice

df 

#### Análisis Descriptivo
Si bien definos centrarnos solamente en variables continuas, a continuación se hará una visuliazación y análisis multivariable, relacionando valores y categorías de manera que se obtengan los siguientes resultados:

- Hay un incremento en las ventas conforme aumenta el presupuesto en TV y Radio
- Hay una concentración mayor en los valores de costos en TV y las ventas 
- La jerarquía de la variable influencer no influyen en las ventas

In [ ]:
fig, ax = plt.subplots(1,3, figsize=(20,5))

sns.scatterplot(data=df, x="tv", y="sales", hue="influencer", ax=ax[0])
ax[0].grid("on")
ax[0].set_xlabel("presupuesto en TV")
ax[0].set_ylabel("ingresos")
ax[0].legend(title="Nivel de Influencer")

sns.scatterplot(data=df, x="radio", y="sales", hue="influencer", ax=ax[1])
ax[1].grid("on")
ax[1].set_xlabel("presupuesto en Radio")
ax[1].set_ylabel("")
ax[1].legend(title="Nivel de Influencer")

sns.scatterplot(data=df, x="social_media", y="sales", hue="influencer", ax=ax[2]) 
ax[2].grid("on")
ax[2].set_xlabel("presupuesto en Redes Sociales")
ax[2].set_ylabel("")
ax[2].legend(title="Nivel de Influencer")

fig.suptitle("Relación de inversiones publicitarias e ingresos de ventas")
plt.subplots_adjust(wspace=0.2)
plt.show()

#### Análisis Exploratorio de datos

Son un conjunto de métodos que utilizan conceptos estadísticos enfocados en explorar y analizar el comportamiento de los datos donde el objetivo es descubrir patrones, relaciones y estructuras que guíen la implementación de un modelo particular como solución. Algunos puntos clave en un caso de regresión son:

Distribución Normal: su presencia en las variables continuas es fundamentalmente requerida para algunos modelos regresores debido a la estructura que genera a la hora de ajustarse a los datos, existen diferentes formas de estimar una distribución de este tipo, como medias y medianas que coinciden, representaciones gráficas con forma simétrica respecto a la media y acampanada con la mayoría de los valores concetrados, sin colas pesadas o nulas(curtosis mesocúritca, cercana a 0).

Correlación Lineal: valor estadístico que indica la fuerza y dirección de una asociación lineal entre dos variables númericas, es una de las medidas estadísticas más importantes y una correlación lineal puede llegar a explicar la influecia de una variable independiente sobre una dependiente.

Outliers: valores que destacan por su notoria distinción(mayor o menor) a los demás del conjunto. La forma más efectiva de hallar estos últimos es la Regla Empírica, según la cual los datos que se encuentran dentro de 3 desviaciones éstandar con respecto a la media representan el 99.7% de los datos, por lo que podemos concluir con bastante seguridad que los datos cuyos valores Z caen por afuera de este umbral son atípicos, ya que son distintos al 99.7%.


In [ ]:
# obteniendo el nivel de correlación lineal
corr, _ = pearsonr(df["radio"], df["sales"])

df_zscore = df.loc[:,["tv","sales"]]

df_zscore["tv_zscore"] = zscore(df["tv"]).abs()
df_zscore["sales_zscore"] = zscore(df["sales"]).abs()

# obteniendo los valores atípicos mediante la prueba empírica
outliers = df_zscore.loc[(df_zscore["tv_zscore"] > 3) | (df_zscore["sales_zscore"] > 3),:]

# obteniendo el valor de curtósis de ambas variables
curtosis_x = kurtosis(df["tv"])
curtosis_y = kurtosis(df["sales"])

print("-----------------------------------------------------------")
print(f"correlación de pearson: {round(corr, 2)}")
print(f"outliers(cantidad): {outliers.shape[0]}")
print(f"curtósis en variable X: {round(curtosis_x, 1)} | curtósis en variable Y: {round(curtosis_y, 1)}")
print("-----------------------------------------------------------")

#### Pruebas de normalidad
Los métodos estadísticos mencionados anteriormente son ampliamente utilizados y dan una base sólida para diagnosticar, sin embargo, las siguientes pruebas de hipótesis proporcionan una evaluación más precisa y son menos propensas a errores de confirmación

Shapiro-Wilk: Se basa en la correlación entre los valores del conjunto y una versión de estos que representan un respectivo de nivel de desviación éstandar, que se denominan valores Z o estandarizados.

Kolmogorov-Smirnov: Compara la función de distribución acumulativa(CDF) del conjunto de valores con la de una distribución normal.

Ambas pruebas hacen uso del valor P para concluir su hipótesis nula(los datos siguen una distribución normal asimilada) o su hipótesis alternativa(caso contrario) con un valor P muy cercano, igual o menor a 0.05.

In [ ]:
# obteniendo el valor P de la prueba de Kolmogorov-Smirnov de ambas variables 
_, p_value_var_x = kstest(df["tv"], "norm")
_, p_value_var_y = kstest(df["sales"], "norm")

# generando una figura múltiple de distribución (varibale X vs variable Y)
fig, ax = plt.subplots(2,2, figsize=(20,6))

sns.histplot(data=df, x="tv", kde=True, ax=ax[0,0])
ax[0,0].set_title("Histograma en Variable X")
ax[0,0].set_xlabel("")
ax[0,0].set_ylabel("")

sns.boxplot(data=df, x="tv", ax=ax[0,1])
ax[0,1].set_title("Gráfico de caja en Variable X")
ax[0,1].set_xlabel("")
ax[0,1].set_ylabel("")

sns.histplot(data=df, x="sales", kde=True, ax=ax[1,0])
ax[1,0].set_title("Histograma en Variable Y")
ax[1,0].set_xlabel("")
ax[1,0].set_ylabel("")

sns.boxplot(data=df, x="sales", ax=ax[1,1])
ax[1,1].set_title("Gráfico de caja en Variable Y")
ax[1,1].set_xlabel("")
ax[1,1].set_ylabel("")

plt.subplots_adjust(hspace=0.5)
plt.suptitle("Figura de distribución (variable X vs variable Y)")
plt.show()

#### Retorno de inversión y modelo lineal 
Luego de haber analizado la relación lineal y distribuciones de ambas variables continuas y obtener resultados como una correlación lineal igual 0.87, distribuciones platicúrticas(más planas que una gaussiana) y nulos valores atípicos se procede a generar el modelo lineal y ponerlo en práctica, pero no con el conjunto de datos general sino que se seleccionarán estratégicamente a los que obtuvieron un mayor beneficio, es decir, que utilizaron costos menores y obtuvieron el mismo o mayor valor de ingresos en ventas.

In [5]:
# calculando el retorno de inversión en cada dato
df["ROI"] = (df["sales"] - df["tv"]) / df["tv"]

# comparando puntos con ingresos similares pero diferentes costos
umbral_roi = df["ROI"].quantile(0.2) # calcula el percentil 20
df_model = df[df["ROI"] >= umbral_roi].copy()

var_x = df_model["tv"].values.reshape((-1,1)) # variable independiente
var_y = df_model["sales"] # variable objetivo

# entrenando el modelo con los datos seleccionados (con un ROI 20% superior)
linear_regression = LinearRegression()
linear_regression.fit(var_x, var_y)

# generando 20 nuevas predicciones de datos aleatorios
objects = df["tv"].sample(n=20).values.reshape((-1,1))

predicts = linear_regression.predict(objects)

#### Dashboard que refleja los insigths obtenidos y el ajuste del modelo lineal sobre los datos

In [ ]:
app = dash.Dash(__name__)

app.layout = html.Div(id="body",className="e2_body",children=[
    html.H1("Marketing sobre Ventas",id="title",className="e2_title"),
    html.Div(id="dashboard",className="e2_dashboard",children=[
        html.Div(id="column-1",className="e2_column_1",children=[
            dcc.Dropdown(id="dropdown",className="e2_dropdown",
                        options=[
                            {"label":"Costos en promociones de TV","value":"tv"},
                            {"label":"Ingresos de ventas","value":"sales"}
                        ],
                        value="tv",
                        multi=False,
                        clearable=False),
            html.Div(className="e2_div_graphs",children=[
                dcc.Graph(id="graph-1",className="e2_graphs",figure={}), 
                dcc.Graph(id="graph-2",className="e2_graphs",figure={})
            ])
        ]),
        html.Div(id="column-2",className="e2_column_2",children=[
            html.H2("Valor-P (Kolmogorov)",id="p_values_title",className="e2_p_values_title"),
            html.Div(id="p_values",className="e2_stats_div",children=[
                html.Div(id="p_value_var_x",className="e2_stats",children=[html.P(f"Variable X: {round(p_value_var_x, 2)}",style={"font-size":"1em"})]),
                html.Div(id="p_value_var_y",className="e2_stats",children=[html.P(f"Variable Y: {round(p_value_var_y, 2)}",style={"font-size":"0.98em"})])
            ]),
            html.Div(f"Correlación lineal: {round(corr,2)}",className="e2_corr",id="corr"),
            dcc.Graph(id="graph-3",className="e2_graph_3",figure={})
        ])
    ])
])

@app.callback(
    [Output(component_id="graph-1",component_property="figure"),
    Output(component_id="graph-2",component_property="figure"),
    Output(component_id="graph-3",component_property="figure")],
    [Input(component_id="dropdown",component_property="value")]
)

def update_dash(slct_var):
    
    mean = df[slct_var].mean()
    median = df[slct_var].median()
    
    var_title = "Campaña publicitaria en TV ($)"
    
    if slct_var == "Sales":
        var_title = "Ventas ($)"
    else:
        var_title = var_title
        
    scatter_radio = go.Figure()
    scatter_radio.add_trace(go.Scatter(x=df[slct_var], y=df["radio"], mode="markers", marker_color="blue"))
    scatter_radio.update_layout(title="Relación con el medio Radio", xaxis_title=var_title, yaxis_title="Campaña publicitaria en Radio ($)")
    
    histplot = go.Figure(go.Histogram(x=df[slct_var], name="Distribución"))
    histplot.add_trace(go.Scatter(x=[mean,mean], y=[0,100], mode="lines+markers", marker_color="red", name="Media"))
    histplot.add_trace(go.Scatter(x=[median,median], y=[0,100], mode="lines+markers", marker_color="green", name="Mediana"))
    histplot.update_layout(title="Histograma", xaxis_title=var_title, yaxis_title=" ")
    
    linear_regression = go.Figure()
    linear_regression.add_trace(go.Scatter(x=df["tv"], y=df["sales"], mode="markers", marker_color="blue", name="Ventas históricas"))
    linear_regression.add_trace(go.Scatter(x=objects.reshape(-1), y=predicts, mode="lines+markers", marker_color="red", name="Ventas estimadas"))
    linear_regression.update_layout(title="Frontera de eficiencia de inversión publicitaria", xaxis_title="Campaña publicitaria en TV ($)", yaxis_title="Ventas ($)")

    return scatter_radio, histplot, linear_regression

if __name__ == "__main__":
    app.run_server(debug=False)

#### Error de bías
También denominado sesgo, es la diferencia entre la predicción esperada del modelo y los valores verdaderos, esto significa una alteración en la dirección de los resultados. Los algoritmos paramétricos generalmente tienen un alto bías que los hace rápidos de aprender y más faciles de entender, pero también menos flexibles y a su vez tienen un menor rendimiento predictivo en problemas complejos. Los algoritmos de alto bías sugieren más suposiciones sobre la forma de la función objetivo, ejemplos: Regresión Lineal, Regresión Logística, Naive Bayes y algoritmos de series temporales. En este ejemplo el algoritmo utilizado asume principalmente que el incremento o decremento de una variable influye a otra, lo cual es completamente acertado según el contexto y los patrones del conjunto.

Se puede observar en el último gráfico del dashboard algunos valores que obtuvieron un resultados similares pero habiendo utilizado un menor presupuesto, este efecto que fue definido en el anterior párrafo tiene la característica de tener un origen predecible y en el problema visto puede deberse a una inversión más participativa y equitativa con la Radio(costos menores) lo que ayudó al éxito financiero de algunos casos. En conclusión, en un caso económico como este un valor correlativo de ese nivel deja en claro una recompensa cuando el marketing se invierte en la TV y el hecho de que algunos datos representen ingresos con menores gastos a comparación de otros lleva a conclusiones beneficiosas en cuanto a la hora de repartir los gastos en publicidad. El modelo de regresión lineal puede servir como guía a la hora de especular sobre el retorno de invrsión, dándole al valor de preupuesto su debido valor de ingreso.